In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/StockForecasting/Stock-Forecast')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from functions import *
os.chdir('/content/drive/MyDrive/dataForStockForecast')

In [3]:
help(preprocess_data_via_close_values)

Help on function preprocess_data_via_close_values in module functions:

preprocess_data_via_close_values(pickle_file_path, num_features=32, min_change=1e-05, shuffle=True, split=True, test_size=0.2, random_state=42)



In [4]:
def models_predict(model_list, x_values):
  preds = np.zeros((x_values.get_data().shape[0], ))
  for mdl in  models_list:
    preds += mdl.predict(x_values)
  return preds/len(model_list)

In [5]:
pickle_file_path = 'training_set.pkl'
x_train, x_valid, y_train, y_valid = preprocess_data_via_close_values(pickle_file_path)

Columns of each dataframe Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
Shape of each array in the list (2169, 32)
Label Classes ['decrease' 'increase' 'no big change']
Data available for each label - increase, decrease and no big change 2093923 2123253 120824
Combined array shape -  (4338000, 32)
Combined labels shape -  (4338000,)
Shuffled data shape -  (4338000, 32)


In [ ]:
import xgboost
xg_train = xgboost.DMatrix(x_train, label=y_train)
xg_valid = xgboost.DMatrix(x_valid)
params = {'objective':'multi:softmax', 'eval_metric':'merror', 'num_class':'3', 'device':'cuda', 'eta':'0.1', 'max_depth':'8'}

In [ ]:
del(x_train)
del(x_valid)

In [ ]:
from tqdm import tqdm
models_list = []
for i in tqdm(range(1)):
  mdl = xgboost.train(params, xg_train, num_boost_round=120)
  models_list.append(mdl)

100%|██████████| 1/1 [00:16<00:00, 16.61s/it]


In [ ]:
gc.collect()

30

In [ ]:
train_preds = models_predict(models_list, xg_train)

In [ ]:
_, _, _, _, _ = metric_calculations(train_preds, y_train, set_ = "Train Set")

For Train Set
  Accuracy: 52.88975334255417,
  Precision: 52.354874918964065,
  Recall: 56.89776044477718,
  Specificity: 48.937211338701644,
  F1score: 54.5318677455742


In [ ]:
valid_preds = models_predict(models_list, xg_valid)

In [ ]:
_, _, _, _, _ = metric_calculations(valid_preds, y_valid, set_ = "Validation Set")

For Validation Set
  Accuracy: 50.16516828031351,
  Precision: 49.91619218494807,
  Recall: 53.89030228042615,
  Specificity: 46.477865389643455,
  F1score: 51.827175156790894


In [ ]:
pickle_file_path = 'testing_set1.pkl'
x_test, y_test = preprocess_data_via_close_values(pickle_file_path, split=False)

Columns of each dataframe Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
Shape of each array in the list (2169, 32)
Label Classes ['decrease' 'increase' 'no big change']
Data available for each label - increase, decrease and no big change 2093923 2123253 120824
Combined array shape -  (4338000, 32)
Combined labels shape -  (4338000,)
Shuffled data shape -  (4338000, 32)


In [ ]:
xg_test = xgboost.DMatrix(x_test)
del(x_test)

In [ ]:
test_preds = models_predict(models_list, xg_test)
del(xg_test)

In [ ]:
_, _, _, _, _ = metric_calculations(test_preds, y_test, set_ = "Test Set")

For Test Set
  Accuracy: 52.344836330106034,
  Precision: 51.86898163965803,
  Recall: 56.29536602843278,
  Specificity: 48.4460707258357,
  F1score: 53.991603885723286
